# scratch

In [1]:
import zarr
import numpy as np
from cloudvolume import CloudVolume, Skeleton
import dask.array as da
from ome_zarr.writer import write_image
from ome_zarr.io import parse_url

In [20]:
outpath = f"precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/traces"
info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",
    encoding="raw",
    resolution=[1, 1, 1],  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=[10, 10, 10],  # units are voxels
    volume_size=[20, 20, 20],  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(outpath, info=info, compress=False)
vol.commit_info()
vol.skeleton.meta.commit_info()

In [21]:
import json

fname = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/traces/skeletons/info"
with open(fname) as f:
    data = json.load(f)
    print(data["vertex_attributes"])
    for i, attr in enumerate(data["vertex_attributes"]):
        if attr["id"] == "vertex_types":
            data["vertex_attributes"].pop(i)
            break

with open(fname, "w") as f:
    json.dump(data, f)

[{'id': 'radius', 'data_type': 'float32', 'num_components': 1}, {'id': 'vertex_types', 'data_type': 'uint8', 'num_components': 1}]


In [18]:
store = parse_url(
    "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test_ngff.zarr",
    mode="w",
).store
root = zarr.group(store=store)
write_image(image=dra, group=root, axes="xyz")

python cors_webserver.py -d "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/" -p 9010

In [35]:
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/frags"

vol_im = CloudVolume(ng_path)
vol_im.chunk_size
vol_im.resolution

Vec(500,500,300, dtype=int64)

In [36]:
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/traces"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=vol_im.resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=vol_im.chunk_size,  # units are voxels
    volume_size=vol_im.shape[:3],  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(ng_path, info=info, compress=False)
vol.commit_info()

## Read ome-zarr

In [11]:
import ome_zarr
import ome_zarr.reader
import numpy as np

In [29]:
file_path = "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test_ngff.zarr"
loc = ome_zarr.io.parse_url(file_path, mode="r")
zarr_reader = ome_zarr.reader.Reader(loc).zarr
res0 = zarr_reader.load("0")

In [30]:
full_image_npy = res0.compute()
print(full_image_npy.shape)

(100, 100, 100)


In [18]:
sub_image = res0[:, :, :]
print(sub_image)
sub_image_npy = sub_image.compute()
print(np.unique(sub_image_npy))

dask.array<from-zarr, shape=(100, 100, 100), dtype=int64, chunksize=(50, 50, 100), chunktype=numpy.ndarray>
[0 1]


## Explain vb

In [5]:
from brainlit.algorithms.connect_fragments import explain_viterbrain
import pickle
import zarr

vb_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/viterbrain.pickle"

with open(vb_path, "rb") as handle:
    vb = pickle.load(handle)

explain_viterbrain(vb, [29, 494, 59], [32, 375, 809])

[194, 195]
[68, 69]
4 states
9 coordinates
0: [29, 494, 59] f98 s194


KeyError: 194

In [3]:
vb.nxGraph.nodes[194]

{
    'type': 'fragment',
    'fragment': 98,
    'point1': array([  9,  66, 300]),
    'point2': array([  7,  65, 309]),
    'orientation1': array([-0.21566555, -0.10783277,  0.97049496]),
    'orientation2': array([-0.21566555, -0.10783277,  0.97049496]),
    'image_cost': 45,
    'twin': 195
}

In [12]:
z_lbl = zarr.open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/labels.zarr"
)

In [16]:
z_lbl_ome = zarr.open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/labels_ome.zarr/0/"
)

In [17]:
z_lbl[28, 495, 59]

4638

In [18]:
z_lbl_ome[28, 495, 59]

4638

In [19]:
vb.fragment_path

'/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/fg_ome.zarr/0'

In [25]:
(z_lbl[28:, 495:, 59:] == z_lbl_vb[28:, 495:, 59:]).all()

False

In [24]:
z_lbl_vb = zarr.open(vb.fragment_path)

In [26]:
vb.fragment_path

'/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/fg_ome.zarr/0'